In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Xception

In [2]:
import os
import zipfile

# Path to the zip file you want to unzip
zip_file_path = '/content/drive/MyDrive/deepfake_new.zip'

# Path to the directory where you want to save the unzipped files
output_dir = '/content/drive/MyDrive/New_dataset'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Directory '{output_dir}' created.")

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("Files unzipped and saved successfully!")


Directory '/content/drive/MyDrive/New_dataset' created.
Files unzipped and saved successfully!


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)
np.random.seed(42)


def get_xception_based_model():
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    inputs = tf.keras.Input(shape=(128, 128, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    opt = Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model


In [ ]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    img = tf.cast(img, tf.float32) / 255.0
    return img

def predict_with_threshold(model, image_path, threshold=0.5):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]

    print(f"Raw Prediction Score: {prediction:.4f}.")

    if prediction >= threshold:
        pred_class = 'real'
    else:
        pred_class = 'fake'

    print(f"Predicted Class ({pred_class}), Probability - {prediction:.4f}.")

In [ ]:

if __name__=="__main__":
    xception_model = get_xception_based_model()


    train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=10.,
      width_shift_range=0.1,
      height_shift_range=0.1)

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/training',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    validate_generator = validation_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/test',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    epochs = 20

    history = xception_model.fit(train_generator, epochs=epochs, validation_data=validate_generator)

    xception_model.save('best_xception_model.h5')

    # # Testing
    # image_path = '<YOUR_IMAGE_PATH>'
    # predict_with_threshold(xception_model, image_path)

Found 1437 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Epoch 1/20
6/6 [==============================] - 390s 60s/step - loss: 0.7060 - accuracy: 0.5170 - val_loss: 0.6865 - val_accuracy: 0.5430
Epoch 2/20
6/6 [==============================] - 31s 5s/step - loss: 0.6697 - accuracy: 0.5825 - val_loss: 0.6672 - val_accuracy: 0.6142
Epoch 3/20
6/6 [==============================] - 36s 6s/step - loss: 0.6505 - accuracy: 0.6180 - val_loss: 0.6516 - val_accuracy: 0.5977
Epoch 4/20
6/6 [==============================] - 31s 5s/step - loss: 0.6209 - accuracy: 0.6590 - val_loss: 0.6541 - val_accuracy: 0.6076
Epoch 5/20
6/6 [==============================] - 31s 5s/step - loss: 0.5793 - accuracy: 0.7035 - val_loss: 0.6387 - val_accuracy: 0.6474
Epoch 6/20
6/6 [==============================] - 33s 6s/step - loss: 0.5733 - accuracy: 0.7022 - val_loss: 0.6204 - val_accuracy: 0.6689
Epoch 7/20
6/6 [==============================] - 33s 5s/step - loss: 0.5528 - accuracy

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# prompt: give me a code to load my ml model

xception_model = tf.keras.models.load_model('/content/drive/MyDrive/model.h5')


In [ ]:
 # Testing

image_path = '/content/drive/MyDrive/Dataset_new/dataset/test/real/real_00780.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/Dataset_new/dataset/test/real/real_00787.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/Dataset_new/dataset/test/fake/mid_179_1111.jpg'
predict_with_threshold(xception_model, image_path)

image_path = '/content/drive/MyDrive/Dataset_new/dataset/test/fake/mid_182_1111.jpg'
predict_with_threshold(xception_model, image_path)


1/1 [==============================] - 0s 21ms/step
Raw Prediction Score: 0.3366.
Predicted Class (fake), Probability - 0.3366.
1/1 [==============================] - 0s 20ms/step
Raw Prediction Score: 0.0813.
Predicted Class (fake), Probability - 0.0813.
1/1 [==============================] - 0s 20ms/step
Raw Prediction Score: 0.9925.
Predicted Class (real), Probability - 0.9925.
1/1 [==============================] - 0s 30ms/step
Raw Prediction Score: 0.8693.
Predicted Class (real), Probability - 0.8693.


In [5]:

# Evaluate the model on the test dataset
test_generator = validation_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/New_dataset/dataset/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

test_loss, test_accuracy = xception_model.evaluate(test_generator)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save the model
xception_model.save('/content/drive/MyDrive/final_model.h5')

Found 604 images belonging to 2 classes.
19/19 [==============================] - 8s 393ms/step - loss: 0.5707 - accuracy: 0.7086
Test Loss: 0.5707
Test Accuracy: 0.7086


In [6]:
# Save the model
xception_model.save('/content/drive/MyDrive/final_model.h5')

**RESNET**

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)
np.random.seed(42)

def create_base_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    return base_model

def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    img = tf.cast(img, tf.float32) / 255.0
    return img

def predict_with_threshold(model, image_path, threshold=0.5):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img).flatten()[0]  # Extract the first element

    print(f"Raw Prediction Score: {prediction:.4f}.")

    if prediction >= threshold:
        pred_class = 'real'
    else:
        pred_class = 'fake'

    print(f"Predicted Class ({pred_class}), Probability - {prediction:.4f}.")

def get_resnet_based_model():
    base_model = create_base_model()

    inputs = tf.keras.Input(shape=(128, 128, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
    outputs = tf.keras.layers.Dense(units=1, activation='linear')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    opt = Adam(learning_rate=0.0001)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

    return model

In [ ]:
if __name__=="__main__":
    resnet_model = get_resnet_based_model()

    # Optionally uncomment the next line to visualize the architecture summary
    # resnet_model.summary()

    train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=10.,
      width_shift_range=0.1,
      height_shift_range=0.1)

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/training',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    validate_generator = validation_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/test',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    epochs = 20

    history = resnet_model.fit(train_generator, epochs=epochs, validation_data=validate_generator)

    resnet_model.save('best_resnet_model.h5')

Found 1437 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Epoch 1/20
6/6 [==============================] - 80s 6s/step - loss: 1.2279 - mean_absolute_percentage_error: 378171968.0000 - val_loss: 0.3279 - val_mean_absolute_percentage_error: 111692576.0000
Epoch 2/20
6/6 [==============================] - 30s 5s/step - loss: 0.3928 - mean_absolute_percentage_error: 231090272.0000 - val_loss: 0.2508 - val_mean_absolute_percentage_error: 241139584.0000
Epoch 3/20
6/6 [==============================] - 31s 6s/step - loss: 0.2553 - mean_absolute_percentage_error: 261860176.0000 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 202076656.0000
Epoch 4/20
6/6 [==============================] - 36s 6s/step - loss: 0.2584 - mean_absolute_percentage_error: 233523824.0000 - val_loss: 0.2640 - val_mean_absolute_percentage_error: 308699072.0000
Epoch 5/20
6/6 [==============================] - 31s 5s/step - loss: 0.2583 - mean_absolute_percentage_error: 247944256.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
    # Testing
    image_path = '/content/drive/MyDrive/dataset/face_pred/check.jpg'
    predict_with_threshold(resnet_model, image_path)

        # Testing
    image_path = '/content/drive/MyDrive/dataset/face_pred/check2.jpg'
    predict_with_threshold(resnet_model, image_path)

        # Testing
    image_path = '/content/drive/MyDrive/dataset/face_pred/check3.jpg'
    predict_with_threshold(resnet_model, image_path)

        # Testing
    image_path = '/content/drive/MyDrive/dataset/face_pred/check4.jpg'
    predict_with_threshold(resnet_model, image_path)

1/1 [==============================] - 1s 546ms/step
Raw Prediction Score: 0.5499.
Predicted Class (real), Probability - 0.5499.
1/1 [==============================] - 0s 27ms/step
Raw Prediction Score: 0.5527.
Predicted Class (real), Probability - 0.5527.
1/1 [==============================] - 0s 32ms/step
Raw Prediction Score: 0.5479.
Predicted Class (real), Probability - 0.5479.
1/1 [==============================] - 0s 26ms/step
Raw Prediction Score: 0.5497.
Predicted Class (real), Probability - 0.5497.


**EFFICIENT NET**




In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)
np.random.seed(42)

def create_base_model():
    base_model = EfficientNetB0(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
    return base_model

def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    img = tf.cast(img, tf.float32) / 255.0
    return img

def predict_with_threshold(model, image_path, ground_truth=None, threshold=0.5):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    probabilities = model.predict(img)

    print(f"Probability: {probabilities.flatten()[0]:.4f}")

    if ground_truth is not None:
        accuracy = (probabilities.flatten()[0] > threshold) == bool(ground_truth)
        print(f"Accuracy: {accuracy}")

    if probabilities.flatten()[0] >= threshold:
        pred_class = 'real'
    else:
        pred_class = 'fake'

    print(f"Predicted Class ({pred_class})")



In [ ]:
if __name__=="__main__":
    efficientnet_model = get_efficientnet_based_model()

    # Optionally uncomment the next line to visualize the architecture summary
    # efficientnet_model.summary()

    train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=10.,
      width_shift_range=0.1,
      height_shift_range=0.1)

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/training',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    validate_generator = validation_datagen.flow_from_directory(
      directory='/content/drive/MyDrive/dataset/test',
      target_size=(128, 128),
      batch_size=256,
      class_mode='binary')

    epochs = 20

    history = efficientnet_model.fit(train_generator, epochs=epochs, validation_data=validate_generator)

    efficientnet_model.save('best_efficientnet_model.h5')



Found 1437 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Epoch 1/20
6/6 [==============================] - 65s 5s/step - loss: 0.4752 - mean_absolute_percentage_error: 270686976.0000 - val_loss: 0.2710 - val_mean_absolute_percentage_error: 177537648.0000
Epoch 2/20
6/6 [==============================] - 30s 5s/step - loss: 0.2633 - mean_absolute_percentage_error: 248255296.0000 - val_loss: 0.2505 - val_mean_absolute_percentage_error: 240047520.0000
Epoch 3/20
6/6 [==============================] - 30s 5s/step - loss: 0.2509 - mean_absolute_percentage_error: 234101440.0000 - val_loss: 0.2636 - val_mean_absolute_percentage_error: 308203616.0000
Epoch 4/20
6/6 [==============================] - 34s 6s/step - loss: 0.2503 - mean_absolute_percentage_error: 249619872.0000 - val_loss: 0.2508 - val_mean_absolute_percentage_error: 263762144.0000
Epoch 5/20
6/6 [==============================] - 30s 5s/step - loss: 0.2497 - mean_absolute_percentage_error: 250752736.0000

In [ ]:
# Testing
image_path = '/content/drive/MyDrive/dataset/face_pred/check.jpg'
predict_with_threshold(efficientnet_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check2.jpg'
predict_with_threshold(efficientnet_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check3.jpg'
predict_with_threshold(efficientnet_model, image_path)

image_path = '/content/drive/MyDrive/dataset/face_pred/check4.jpg'
predict_with_threshold(efficientnet_model, image_path)

1/1 [==============================] - 0s 48ms/step
Probability: 0.5430
Predicted Class (real)
1/1 [==============================] - 0s 54ms/step
Probability: 0.5422
Predicted Class (real)
1/1 [==============================] - 0s 44ms/step
Probability: 0.5420
Predicted Class (real)
1/1 [==============================] - 0s 51ms/step
Probability: 0.5418
Predicted Class (real)
